# Steam Library Analyzer

## 1.1 Data sets: Get The Games from the Steam Library

In [1]:
import requests
import json
import time
import pandas as pd

import re

In [2]:
# Load the steam credentials JSON file 
with open('steam_credentials.json') as json_file:
    credentials = json.load(json_file)
    
api_key = credentials.get('api_key')
steam_id = credentials.get('steam_id')  

In [3]:
# Get data from my Steam library
if(api_key != None  and steam_id != None):
    games_endpoint = f"https://api.steampowered.com/IPlayerService/GetOwnedGames/v1/?key={api_key}&steamid={steam_id}"

response_games = requests.get(games_endpoint)

In [4]:
# Response validation
if response_games.status_code == 200:
    data_games = response_games.json()
    data_games = json.dumps(data_games, indent=4)
    
    with open("steam_library_data.json", "w") as outfile:
        outfile.write(data_games)

else:
    print(f"Error: {response.status_code}")

## 1.2 Data sets: Get Played Time and Name of the Games

In [5]:
# Load the library data JSON file
with open('steam_library_data.json') as json_file:
    data_games = json.load(json_file)

In [31]:
# Get the name of the games 
errors = 0
all_games_data = []

# Iterate over "steam_library_data" to find the name of the applications
for game in data_games["response"]["games"]:
    
    appid = game['appid']
    app_details_endpoint = f"https://store.steampowered.com/api/appdetails/?appids={appid}"
    response_app_details = requests.get(app_details_endpoint)
    
    if response_app_details.status_code == 200:
        
        try:
            data_app_details = response_app_details.json()
            game_name = data_app_details[str(appid)]['data']['name']
            playtime_forever = game['playtime_forever']
            print(f"Name: {game_name}, ID: {appid}, Played Time: {game['playtime_forever']} minutos")
            
            game_data = {
                "Name": game_name,
                "ID": appid,
                "Playtime_forever": playtime_forever
            }
            all_games_data.append(game_data)
    
        except:
            print(f"Error: {game_name} - {response_app_details.status_code}")
            pass
        
# Save games information
with open("played_time.json", "w") as outfile:
    json.dump(all_games_data, outfile, indent=4)

print(f"Process Completed. Errors {errors}")       

Name: DOOM 3, ID: 9050, Played Time: 142 minutos
Name: DOOM 3 Resurrection of Evil, ID: 9070, Played Time: 0 minutos
Name: DOOM 3, ID: 208200, Played Time: 0 minutos
Name: Portal, ID: 400, Played Time: 245 minutos
Name: The Witcher: Enhanced Edition Director's Cut, ID: 20900, Played Time: 0 minutos
Name: Prince of Persia: Warrior Within™, ID: 13500, Played Time: 0 minutos
Name: Prince of Persia: The Two Thrones™, ID: 13530, Played Time: 0 minutos
Name: Prince of Persia®: The Sands of Time, ID: 13600, Played Time: 0 minutos
Name: Prince of Persia®, ID: 19980, Played Time: 0 minutos
Name: Dead Space (2008), ID: 17470, Played Time: 37 minutos
Name: LEGO® Star Wars™ - The Complete Saga, ID: 32440, Played Time: 101 minutos
Name: Left 4 Dead 2, ID: 550, Played Time: 2244 minutos
Name: Prince of Persia: The Forgotten Sands™, ID: 33320, Played Time: 472 minutos
Name: Super Meat Boy, ID: 40800, Played Time: 751 minutos
Name: Dead Space™ 2, ID: 47780, Played Time: 0 minutos
Name: Portal 2, ID: 6

Name: ScourgeBringer, ID: 1037020, Played Time: 747 minutos
Name: Devolver Bootleg, ID: 1066260, Played Time: 256 minutos
Name: Kathy Rain, ID: 370910, Played Time: 671 minutos
Name: Inscryption, ID: 1092790, Played Time: 3570 minutos
Name: DOOM Eternal, ID: 782330, Played Time: 0 minutos
Name: Halo: The Master Chief Collection, ID: 976730, Played Time: 395 minutos
Name: Shady Part of Me, ID: 1116580, Played Time: 3 minutos
Name: Curse of the Dead Gods, ID: 1123770, Played Time: 0 minutos
Name: STAR WARS Jedi: Fallen Order™, ID: 1172380, Played Time: 1864 minutos
Name: Red Dead Redemption 2, ID: 1174180, Played Time: 0 minutos
Name: Moncage, ID: 1195290, Played Time: 160 minutos
Name: Hades, ID: 1145360, Played Time: 0 minutos
Name: Exit the Gungeon, ID: 1209490, Played Time: 65 minutos
Name: Potion Craft: Alchemist Simulator, ID: 1210320, Played Time: 708 minutos
Name: Unravel Two, ID: 1225570, Played Time: 603 minutos
Name: Dead Space™ 3, ID: 1238060, Played Time: 0 minutos
Name: Cyr

In [28]:
with open('played_time.json') as json_played_time:
    played_time_file = json.load(json_played_time)  
# print(played_time_file)

## 1.3 Data sets: Get the howlongtobeat data

In [32]:
games = 0
for game in played_time_file:  
    game_name = game["Name"]
    game_name1 = re.sub(r'[^a-zA-Z0-9\s\:\.\-\,]', '', game_name)

    beat_time_data = f"https://hltb-api.vercel.app/api?name={game_name1}"
    beat_time_response = requests.get(beat_time_data)

    if beat_time_response.status_code == 200:
        beat_time_data = beat_time_response.json()
        try:
            print("Name:", beat_time_data[0]['name'], " - Beat Time:", beat_time_data[0]['gameplayMain'])
            games += 1
            #if(beat_time_data[0]['gameplayCompletionist'] != 0):
            #    print("Name:", beat_time_data[0]['name'], " - Beat Time:", beat_time_data[0]['gameplayCompletionist'])
                
            #else:
            #    print("Name:", beat_time_data[0]['name'], " - Beat Time:", beat_time_data[0]['gameplayMain'])
            #    games += 1
        except:
            
            print("ERROR:", game_name1)

Name: Doom 3  - Beat Time: 11
Name: Doom 3: Resurrection of Evil  - Beat Time: 4
Name: Doom 3  - Beat Time: 11
Name: Portal  - Beat Time: 3
Name: The Witcher  - Beat Time: 35
Name: Prince of Persia: Warrior Within  - Beat Time: 12
Name: Prince of Persia: The Two Thrones  - Beat Time: 10
Name: Prince of Persia: The Sands of Time  - Beat Time: 9
Name: Prince of Persia  - Beat Time: 11
Name: Dead Space  - Beat Time: 12
Name: LEGO Star Wars: The Complete Saga  - Beat Time: 13
Name: Left 4 Dead 2  - Beat Time: 10
Name: Prince of Persia: The Forgotten Sands  - Beat Time: 8
Name: Super Meat Boy  - Beat Time: 10
Name: Dead Space 2  - Beat Time: 9
Name: Portal 2  - Beat Time: 9
Name: Terraria  - Beat Time: 52
Name: The Witcher 2: Assassins of Kings  - Beat Time: 24
Name: Alice: Madness Returns  - Beat Time: 14
Name: Limbo  - Beat Time: 4
Name: The Binding of Isaac  - Beat Time: 11
Name: The Walking Dead  - Beat Time: 1
Name: Batman: Arkham Asylum  - Beat Time: 12
Name: Castle Crashers  - Beat T

Name: Beholder  - Beat Time: 6
Name: Cave Story's Secret Santa  - Beat Time: 1
Name: A Plague Tale: Requiem  - Beat Time: 17
